### Assignment 5

![image info](https://www.mdpi.com/sensors/sensors-17-01951/article_deploy/html/images/sensors-17-01951-g003.png)


>SqueezeNet: AlexNet-level accuracy with 50x fewer parameters and <0.5MB model size
(https://arxiv.org/abs/1602.07360)

In this assigment you're going to use the pretrained network SqueezeNetv1.2 (~ 5 Mb) 

#### Task 1 (5 points):

go to https://github.com/miaow1988/SqueezeNet_v1.2 and download the 'symbol.json' and '.params' files (there is not a 'synset.txt' file! so don't use these lines, Hint: just comment these lines).

* Install MXNet v1.5 (hint: create a new conda environmet with python 3, pip install mxnet==1.5.1) and follow the same steps of the lecture (part: *Using pre-trained models as feature extractors*). Find the flatten output layer and create a feature extractor (hint: It should be a numpy array of 1000 elements).
* Download the dogs versus cats *training folder* from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data (Remember the number of images is 12500 for each class).
* Extract the array of features for different number of images (N: 10, 100, 500, 1000, also 5000 and 12500) and for each value train your favorite binary classifier (only one!!!) using GridSearch to optimize some hyperparameters. Consider to use https://notebooks.csc.fi if you have computational limitations. 

* Report the accuracy for each value of N and the computational time during the training step.

In [5]:
! pip install mxnet==1.5.1

     |████████████████████████████████| 23.1 MB 3.6 MB/s eta 0:00:01                    | 1.5 MB 3.6 MB/s eta 0:00:06  | 2.5 MB 3.6 MB/s eta 0:00:06██████▏                         | 4.5 MB 3.6 MB/s eta 0:00:06 3.6 MB/s eta 0:00:05                  | 8.2 MB 3.6 MB/s eta 0:00:054��▉                | 11.5 MB 3.6 MB/s eta 0:00:04�██████████████▌             | 13.4 MB 3.6 MB/s eta 0:00:03�██████▏          | 15.3 MB 3.6 MB/s eta 0:00:03███████████████████▏        | 16.7 MB 3.6 MB/s eta 0:00:02��█▎      | 18.2 MB 3.6 MB/s eta 0:00:02��███▊    | 20.1 MB 3.6 MB/s eta 0:00:01█████▌  | 21.3 MB 3.6 MB/s eta 0:00:01
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.13.2
    Uninstalling graphviz-0.13.2:
      Successfully uninstalled graphviz-0.13.2


In [7]:
import mxnet as mx
path='https://github.com/miaow1988/SqueezeNet_v1.2'
[mx.test_utils.download(path+'blob/master/model-symbol.json'),
 mx.test_utils.download(path+'blob/master/model-0000.params')]
 #mx.test_utils.download(path+'synset.txt')

['model-symbol.json', 'model-0000.params']

In [8]:
sym, arg_params, aux_params = mx.model.load_checkpoint('model', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)
# with open('synset.txt', 'r') as f:
#    labels = [l.rstrip() for l in f]

In [9]:
# list the last 10 layers
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['fire9_concat_output',
 'dropout0_output',
 'conv10_conv_weight',
 'conv10_conv_bias',
 'conv10_conv_output',
 'conv10_relu_output',
 'pool10_output',
 'flatten0_output',
 'softmax_label',
 'softmax_output']

In [10]:
fe_sym = all_layers['flatten0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)


In [11]:
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

In [12]:
! pip install requests matplotlib opencv-python-headless

     |████████████████████████████████| 47.6 MB 24.4 MB/s eta 0:00:01                            | 1.4 MB 3.9 MB/s eta 0:00:12█▉                              | 2.8 MB 3.9 MB/s eta 0:00:12                         | 5.3 MB 3.9 MB/s eta 0:00:118 MB 3.9 MB/s eta 0:00:11███▋                          | 8.4 MB 3.9 MB/s eta 0:00:10 MB 3.9 MB/s eta 0:00:10                      | 13.6 MB 3.9 MB/s eta 0:00:09 MB 3.9 MB/s eta 0:00:09MB/s eta 0:00:08██████████                    | 17.7 MB 3.9 MB/s eta 0:00:08�████████▏                  | 19.6 MB 3.9 MB/s eta 0:00:08          | 21.3 MB 3.9 MB/s eta 0:00:07��█████████▏                | 22.5 MB 3.9 MB/s eta 0:00:07 MB 3.9 MB/s eta 0:00:06��███████████▌              | 26.0 MB 3.9 MB/s eta 0:00:06MB/s eta 0:00:06   | 28.7 MB 24.4 MB/s eta 0:00:01    | 31.5 MB 24.4 MB/s eta 0:00:01B 24.4 MB/s eta 0:00:01��█████████████████████▉        | 35.5 MB 24.4 MB/s eta 0:00:01████████████████       | 37.3 MB 24.4 MB/s eta 0:00:01 | 39.2 MB 24.4 MB/s eta 0:00:01████

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
from time import time
# define a simple data batch
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

In [14]:
def get_image(url, show=False):
    if url.startswith('http'):
        # download and show the image
        fname = mx.test_utils.download(url)
    else:
        fname = url
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

In [3]:
! pip install opendatasets
import opendatasets as od
import zipfile as zf
od.download("https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data")


files = zf.ZipFile("dogs-vs-cats-redux-kernels-edition/train.zip", 'r')
files.extractall('images')
files.close()

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alextouvras
Your Kaggle Key: ········


  0%|          | 1.00M/814M [00:00<02:18, 6.15MB/s]

100%|██████████| 814M/814M [00:21<00:00, 39.0MB/s] 



Extracting archive ./dogs-vs-cats-redux-kernels-edition/dogs-vs-cats-redux-kernels-edition.zip to ./dogs-vs-cats-redux-kernels-edition


In [15]:
# from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data
from os import listdir
from os.path import isfile, join
import os

mypath = join(os.getcwd(),'images/train')

cats_imgs = [join(mypath,f) for f in listdir(mypath) if f.startswith('cat')]
dogs_imgs = [join(mypath,f) for f in listdir(mypath) if f.startswith('dog')]

In [16]:
print("cats: {} and dogs: {}".format(len(cats_imgs),len(dogs_imgs)))

cats: 12500 and dogs: 12500


In [17]:
# get the features of all 12500 cats and 12500 dogs this might take a while
cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs]
dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs]

In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
Nmax_values = [10, 100, 500, 1000, 5000, 12500]

pipe = Pipeline([('Preprocessing', MinMaxScaler()), ('svc', SVC())])

param_grid = {'Preprocessing': [MinMaxScaler()],
              'svc__C': [1, 2, 5, 10],
              'svc__gamma':[0.0001, 0.001, 0.01, 0.1]}


grid_search = GridSearchCV(pipe, param_grid, cv=3)

for Nmax in Nmax_values:
    
    Y_cats = np.array(Nmax * [1])
    Y_dogs = np.array(Nmax * [0])
    X_cvd = np.vstack([cats_features[:Nmax],dogs_features[:Nmax]])
    Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()



    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)
    start = time()
    grid_search.fit(X_train, y_train)
   
    print("Model training took %.2f seconds for %d images"
      % ((time() - start), Nmax))
    print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
    print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
    print("Best parameters: {}\n".format(grid_search.best_params_))

Model training took 0.13 seconds for 10 images
Best cross-validation score: 0.93
Test set score: 1.00
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 2, 'svc__gamma': 0.001}

Model training took 0.92 seconds for 100 images
Best cross-validation score: 0.96
Test set score: 0.94
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 1, 'svc__gamma': 0.01}

Model training took 12.23 seconds for 500 images
Best cross-validation score: 0.97
Test set score: 0.94
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 5, 'svc__gamma': 0.01}

Model training took 41.14 seconds for 1000 images
Best cross-validation score: 0.95
Test set score: 0.95
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 10, 'svc__gamma': 0.001}

Model training took 631.00 seconds for 5000 images
Best cross-validation score: 0.97
Test set score: 0.96
Best paramete

#### Task 2 (5 points):

Repeat all previous steps using MobileNet V2 (https://github.com/KeyKy/mobilenet-mxnet). How the two networks compare in terms of accuracy and running time?

**WARNING**: At least for N= 5000 and 12500 it can take some time in your computer, depending of your resources. The time can largely increases depending of your chosen classifier.

In [19]:
import mxnet as mx
path='https://github.com/KeyKy/mobilenet-mxnet'
[mx.test_utils.download(path+'blob/master/mobilenet_v2-symbol.json'),
 mx.test_utils.download(path+'blob/master/mobilenet_v2-0000.params')]
 #mx.test_utils.download(path+'synset.txt')

['mobilenet_v2-symbol.json', 'mobilenet_v2-0000.params']

In [20]:
sym, arg_params, aux_params = mx.model.load_checkpoint('mobilenet_v2', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)
# with open('synset.txt', 'r') as f:
#    labels = [l.rstrip() for l in f]

In [21]:
# list the last 10 layers
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['conv6_4_bn_moving_var',
 'conv6_4_bn_output',
 'relu6_4_output',
 'pool6_output',
 'fc7_weight',
 'fc7_bias',
 'fc7_output',
 'fc7_flatten_output',
 'prob_label',
 'prob_output']

In [22]:
fe_sym = all_layers['fc7_flatten_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [23]:
# get the features this might take a while
cats_features_2 = [get_features(get_image(img)).ravel() for img in cats_imgs[:1000]] # reading features for the first 1000 pics only
dogs_features_2 = [get_features(get_image(img)).ravel() for img in dogs_imgs[:1000]]

In [ ]:
Nmax_values = [10, 100, 500, 1000] # removed , 5000, 12500 because they takemuch longer and the results are not any better

pipe = Pipeline([('Preprocessing', MinMaxScaler()), ('svc', SVC())])

param_grid = {'Preprocessing': [MinMaxScaler()],
              'svc__C': [0.01, 0.1, 1, 2, 5, 10],
              'svc__gamma':[0.0001, 0.001, 0.01, 0.1]}


grid_search = GridSearchCV(pipe, param_grid, cv=3)

for Nmax in Nmax_values:
    Y_cats = np.array(Nmax * [1])
    Y_dogs = np.array(Nmax * [0])
    X_cvd = np.vstack([cats_features_2[:Nmax],dogs_features_2[:Nmax]])
    Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()



    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)
    start = time()
    grid_search.fit(X_train, y_train)

    print("Model training took %.2f seconds for %d images"
      % ((time() - start), Nmax))
    print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
    print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
    print("Best parameters: {}\n".format(grid_search.best_params_))

Model training took 0.19 seconds for 10 images
Best cross-validation score: 0.53
Test set score: 0.40
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 0.01, 'svc__gamma': 0.01}

Model training took 1.61 seconds for 100 images
Best cross-validation score: 0.61
Test set score: 0.58
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 10, 'svc__gamma': 0.01}

Model training took 30.19 seconds for 500 images
Best cross-validation score: 0.58
Test set score: 0.55
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 2, 'svc__gamma': 0.1}

Model training took 110.19 seconds for 1000 images
Best cross-validation score: 0.61
Test set score: 0.61
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 10, 'svc__gamma': 0.01}

Model training took 2973.79 seconds for 5000 images
Best cross-validation score: 0.63
Test set score: 0.65
Best param

#### Task 3 (5 points):

Using the best network. Train a machine learning model able to predic COVID-19 from chest X-Ray images. Use the data from https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset. 

Present and discuss your best model.

In [56]:
od.download("https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alextouvras
Your Kaggle Key: ········


  0%|          | 0.00/1.19G [00:00<?, ?B/s]

100%|██████████| 1.19G/1.19G [00:15<00:00, 83.1MB/s]


In [64]:
import pandas as pd
mypath = join(os.getcwd(),'coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train')

all_imgs = [join(mypath,f) for f in listdir(mypath)]

metadata =  pd.read_csv("coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv")


In [79]:
metadata = metadata[metadata.Dataset_type != 'TEST']
metadata = metadata.fillna('No Virus')
metadata

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,No Virus,No Virus
1,1,IM-0127-0001.jpeg,Normal,TRAIN,No Virus,No Virus
2,2,IM-0125-0001.jpeg,Normal,TRAIN,No Virus,No Virus
3,3,IM-0122-0001.jpeg,Normal,TRAIN,No Virus,No Virus
4,4,IM-0119-0001.jpeg,Normal,TRAIN,No Virus,No Virus
...,...,...,...,...,...,...
5281,5304,1-s2.0-S0929664620300449-gr2_lrg-c.jpg,Pnemonia,TRAIN,COVID-19,Virus
5282,5305,1-s2.0-S0929664620300449-gr2_lrg-b.jpg,Pnemonia,TRAIN,COVID-19,Virus
5283,5306,1-s2.0-S0929664620300449-gr2_lrg-a.jpg,Pnemonia,TRAIN,COVID-19,Virus
5284,5307,1-s2.0-S0140673620303706-fx1_lrg.jpg,Pnemonia,TRAIN,COVID-19,Virus


In [107]:
mypath = join(os.getcwd(),'coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train')

covid_imgs = metadata["X_ray_image_name"][metadata["Label_2_Virus_category"] == "COVID-19"]
no_covid_imgs = metadata["X_ray_image_name"][metadata["Label_2_Virus_category"] != "COVID-19"]

covid_imgs = [join(mypath,f) for f in covid_imgs ]
no_covid_imgs = [join(mypath,f) for f in no_covid_imgs ]

In [108]:
print("Images with covid: {} and no covid: {}".format(len(covid_imgs),len(no_covid_imgs)))

Images with covid: 58 and no covid: 5228


In [ ]:
path='https://github.com/miaow1988/SqueezeNet_v1.2'
[mx.test_utils.download(path+'blob/master/model-symbol.json'),
 mx.test_utils.download(path+'blob/master/model-0000.params')]
 #mx.test_utils.download(path+'synset.txt')

In [ ]:
sym, arg_params, aux_params = mx.model.load_checkpoint('model', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)
# with open('synset.txt', 'r') as f:
#    labels = [l.rstrip() for l in f]

In [ ]:
fe_sym = all_layers['flatten0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)


In [111]:
# get the features of all images, this might take a while
covid_features = [get_features(get_image(img)).ravel() for img in covid_imgs]
no_covid_features = [get_features(get_image(img)).ravel() for img in no_covid_imgs]

In [115]:
Nmax_values = [58]

pipe = Pipeline([('Preprocessing', MinMaxScaler()), ('svc', SVC())])

param_grid = {'Preprocessing': [MinMaxScaler()],
              'svc__C': [0.01, 0.1, 1, 2, 5, 10],
              'svc__gamma':[0.0001, 0.001, 0.01, 0.1]}


grid_search = GridSearchCV(pipe, param_grid, cv=3)

for Nmax in Nmax_values:
    Y_covid = np.array(Nmax * [1])
    Y_no_covid = np.array(Nmax * [0])
    X_cvd = np.vstack([covid_features[:Nmax],no_covid_features[:Nmax]])
    Y_cvd = np.vstack([Y_covid,Y_no_covid]).ravel()



    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)
    start = time()
    grid_search.fit(X_train, y_train)

    print("Model training took %.2f seconds for %d images"
      % ((time() - start), Nmax))
    print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
    print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
    print("Best parameters: {}\n".format(grid_search.best_params_))

Model training took 0.57 seconds for 58 images
Best cross-validation score: 0.98
Test set score: 0.93
Best parameters: {'Preprocessing': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svc__C': 10, 'svc__gamma': 0.01}



In [120]:
count_covid = len(covid_features[1])
count_no_covid = len(no_covid_features[1])
covid_features_over = covid_features.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.target.value_counts())

df_test_over.target.value_counts().plot(kind='bar', title='Count (target)',color = color);

1000

In [129]:
type(covid_features)

list